In [8]:
# import the necessary packages
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import argparse
import pickle
import h5py
from keras.applications import VGG16


In [9]:
output = "model/"
jobs = 1

In [10]:
db = h5py.File("HDF5file", "r") #open the HDF5 model that was created by feature extraction
i = int(db["labels"].shape[0] * 0.9)

In [11]:
# define the set of parameters that we want to tune then start a
# grid search where we evaluate our model for each value of C
print("[INFO] tuning hyperparameters...")
params = {"C": [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}
model = GridSearchCV(LogisticRegression(max_iter=10000), params, cv=3,
	n_jobs=jobs)

[INFO] tuning hyperparameters...


In [ ]:
model.fit(db["features"][:i], db["labels"][:i])
print("[INFO] best hyperparameters: {}".format(model.best_params_))

In [12]:
trained_model = "model/trained"

# load the VGG16 network
print("[INFO] loading network...")
vgg = VGG16(weights="imagenet", include_top=False)
# load the orientation model
print("[INFO] loading model...")
model = pickle.loads(open(trained_model, "rb").read())

[INFO] loading network...
[INFO] loading model...


In [13]:
labelNames = [(angle) for angle in db["label_names"][i:]]
print(labelNames)

[]


In [14]:
# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(db["features"][i:])
print(classification_report(db["labels"][i:], preds,
	target_names=db["label_names"]))

[INFO] evaluating...
              precision    recall  f1-score   support

   Rotated/0       0.97      0.96      0.97       307
 Rotated/180       0.94      0.95      0.94       294
 Rotated/270       0.88      0.91      0.89       255
  Rotated/90       0.92      0.90      0.91       300

    accuracy                           0.93      1156
   macro avg       0.93      0.93      0.93      1156
weighted avg       0.93      0.93      0.93      1156



In [ ]:
# serialize the model to disk
print("[INFO] saving model...")
f = open("model/trained", "wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()

In [ ]:
# close the database
db.close()